In [1]:
import pandas as pd
import numpy as np

In [30]:
# loading data
df = pd.read_csv("train.csv")

In [31]:
'''# greet the data
print(df.describe())
print("-"*65)
print(df.describe(include=['O']))
print("-"*65)
print(df.info())'''

'# greet the data\nprint(df.describe())\nprint("-"*65)\nprint(df.describe(include=[\'O\']))\nprint("-"*65)\nprint(df.info())'

In [6]:
# create training and testing datasets and actual labels
train = df.copy()
train = train[train.Date <= '2020-05-10']

test = df.copy()
test = test[(test['Date']>='2020-05-11')&(test['Date']<='2020-05-17')]

actuals = test.TargetValue

In [20]:
'''# seems like the data is cleaned
train.Country_Region.unique().tolist()
train.County.unique().tolist()
train.Province_State.unique().tolist()'''

'# seems like the data is cleaned\ntrain.Country_Region.unique().tolist()\ntrain.County.unique().tolist()\ntrain.Province_State.unique().tolist()'

In [32]:
'''# See the training data
print(train.describe())
print("-"*65)
print(train.describe(include=['O']))
print("-"*65)
print(train.info())'''

'# See the training data\nprint(train.describe())\nprint("-"*65)\nprint(train.describe(include=[\'O\']))\nprint("-"*65)\nprint(train.info())'

A couple of ideas for models:
- quntile linear regression from mlinsights
- quantile gradient boosting regressor from sklearn
- quantile regression forests from skargen
- quantile deep net using Keras or Tensorflow

A couple of ideas for the missing values:
- drop 
- impute
- impute with mark on which ones are missing

A couple of ideas for the columns:

- County: drop or keep
- Province_State: drop or keep
- Population: bins or as is
- Date: day of week, day of month, month, day of year

- relationship b/w confirmed and death?

- New columns:Continuent ?

Ways of encoding

...

In [116]:
#Create the Weighted Pinball Loss function

def PLoss(preds, actuals, qt):
    loss = np.maximum((actuals-preds)*qt, (preds-actuals)*(1-qt))
    return loss

def WPLoss(preds05, preds50, preds95, actuals, weights):
    l05 = PLoss(preds05, actuals,0.05)
    l50 = PLoss(preds50, actuals,0.5)
    l95 = PLoss(preds95, actuals,0.95)
    score = sum((l05+l50+l95)/3*weights)/actuals.count()
    return score

In [14]:
# Create label y
y = train.TargetValue

# Clean train and test data
train = train.drop(['Id','TargetValue'],axis=1)
testId = test.Id
test = test.drop(['Id','TargetValue'],axis=1)

In [15]:
#encode and creat X
train_encoded = pd.get_dummies(train)
test_encoded = pd.get_dummies(test)

train_ali, test_ali = train_encoded.align(test_encoded, join='left', axis=1)

X=train_ali

In [16]:
# Split train data into validation and training data
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X, y,random_state=1, test_size=0.25)

Let's try the models with droping the columns with missing values


In [ ]:
#drop columns with missing values 
cols_with_missing = [col for col in train_X.columns 
                                 if train_X[col].isnull().any()]

train_X_noMis = train_X.copy()
val_X_noMis  = val_X.copy()
train_X_noMis = train_X_noMis.drop(cols_with_missing, axis=1)
val_X_noMis  = val_X_noMis.drop(cols_with_missing, axis=1)

In [22]:
# Not sure what the 'quantile' loss function in GBR!

#GradientBoostingRegressor

from sklearn.ensemble import GradientBoostingRegressor

def gbrPred(train_X, val_X, train_y,val_y,qt):
    model = GradientBoostingRegressor(loss='quantile', alpha=qt)
    model.fit(train_X,train_y)
    
    preds = model.predict(val_X)
    return preds

gbr05 = gbrPred(train_X_noMis, val_X_noMis, train_y,val_y,0.05)
gbr50 = gbrPred(train_X_noMis, val_X_noMis, train_y,val_y,0.5)
gbr95 = gbrPred(train_X_noMis, val_X_noMis, train_y,val_y,0.95)

gbr_score = WPLoss(gbr05, gbr50, gbr95, val_y, val_X.Weight)

In [ ]:
# quantile linear regression
#from mlinsights.mlmodel import QuantileLinearRegression

In [ ]:
# Deep nets
